Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [27]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [28]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [29]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [30]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [31]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [32]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [33]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [34]:

def ESstrategy(state: Nim, values):
    selection = np.random.choice(len(values), p=values)
    possible_moves = [
        [i, state._rows[i] if state._k is None else min(state._rows[i], state._k)]
        for i in range(len(state._rows))
        if state._rows[i] != 0
    ]
    if selection == 0:
        ply = max(possible_moves, key=lambda e: e[0])
    elif selection == 1:
        ply = max(possible_moves, key=lambda e: e[1])
    elif selection == 2:  
        ply = min(possible_moves, key=lambda e: e[1])
    elif selection ==3:  
        ply = min(possible_moves, key=lambda e: e[0])

    return Nimply(ply[0], ply[1])

In [35]:
λ = 20
μ = 15
σ = 2
PPsize = 50
n = 4

def fitness(values):
    count = 0
    for _ in range(PPsize):
        nim = Nim(5)
        player = 0
        while nim:
            ply = optimal(nim) if player == 0 else ESstrategy(nim, values)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            count += 1

    return (count*100)/PPsize


def mutation(population):
    offspring = []

    for _ in range(λ):
        p = population[np.random.randint(0, μ)]
        off =(p[0], np.abs(np.random.normal(loc=p[-1], scale=0.2)))
        noise = np.abs(np.random.normal(loc=off[0], scale=off[1])) 
        noise = list(noise / np.sum(noise))
        offspring.append((noise, fitness(noise),off[1]))

    return offspring

def mutation_m(population):
    offspring = []

    for p in population:
        for _ in range(λ // μ):
            parent = population[np.random.randint(0, μ)]
            mutated_probabilities = np.abs(
                np.random.normal(loc=parent[0], scale=0.2))
            mutated_probabilities /= np.sum(mutated_probabilities)
            offspring.append((mutated_probabilities, fitness(mutated_probabilities), parent[-1]))

    return offspring


In [36]:
def ES():
    population = []
    TopFitness = None
    BestPr = None

    for _ in range(μ):
        probabilities = list(np.random.dirichlet(np.ones(n)))
        population.append((probabilities, fitness(probabilities), σ))
    step = 0

    while TopFitness is None or (TopFitness < 50.0 and step < 100):
        offspring = mutation_m(population)
        offspring.extend(population)
        md = sorted(offspring, key=lambda e: e[1])
        if TopFitness is None or md[-1][1] > TopFitness:
            TopFitness = md[-1][1]
            BestPr = md[-1][0]

        population = md[-μ:]
        step += 1

    return BestPr, TopFitness

adaptive_player, rating = ES()

print(f"Best Probabilities is equal to : {adaptive_player}\nWin percentage is equal to:{rating}%")

In [ ]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    return ESstrategy(state, adaptive_player)

## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=8)
INFO:root:status: <1 3 5 7 1>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=4)
INFO:root:status: <0 3 5 3 1>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 5 3 1>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 2 1 3 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 2 0 3 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 2 0 3 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 1 0 3 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 1 0 2 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 1 0 1 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0